In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

In [2]:
def init_browser():
    executable_path = {"executable_path": "C:\\Users\\bethf\\Desktop\\chromedriver"}
    return Browser("chrome", **executable_path, headless=True)

In [3]:
def scrape_data():
    browser = init_browser()

    # Visit NASA
    url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    browser.visit(url)
    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "lxml")

    # Scrape data
    article = soup.find_all(class_="content_title")[0].text.strip()
    news_p = soup.find_all(class_="article_teaser_body")[0].text.strip()
    relativearticleurl = soup.find_all("a")[35]["href"]
    articleurl = 'https://mars.nasa.gov' + relativearticleurl
    relative_image_path = soup.find_all('img')[2]['src']
    nasa_img = 'https://mars.nasa.gov' + relative_image_path
    
    # Visit NASA images
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url)
    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "lxml")

    # Scrape data
    relative_image_path = soup.find_all('img')[3]['src']
    first_image_url = 'https://jpl.nasa.gov' + relative_image_path
    
    relative_image_path = soup.find_all('article',class_="carousel_item")[0]["style"]
    relative_image_path = relative_image_path.replace("background-image: url('",'')
    relative_image_path = relative_image_path.replace("');",'')
    featured_image_url = 'https://jpl.nasa.gov' + relative_image_path
    
    #Visit Mars Weather
    url = "https://twitter.com/marswxreport?lang=en"
    browser.visit(url)
    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "lxml")
    
    # Scrape data
    mars_weather = soup.find_all('p',class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")[0].text.strip()
    
    #Visit Mars Facts
    url = "https://space-facts.com/mars/"

    #Scrape Table
    table = pd.read_html(url)[0]
    mars_facts = table.to_html(header=False,index=False)
    mars_facts = mars_facts.replace('\n','')
    mars_facts
    
    #Loop for Hemispheres
    hemispheres_list = []

    for num in range(0,4):
        hemispheres = {}
        
        #Visit Mars Astropedia
        url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
        browser.visit(url)
        html = browser.html
        soup = bs(html, "lxml")
        
        #Click on hemisphere for full size image
        browser.find_by_css("h3")[num].click()

        html = browser.html
        soup = bs(html, "lxml")

        #Scrape data
        relative_path = soup.find('img',class_="wide-image")['src']
        img = 'https://astrogeology.usgs.gov' + relative_path
        title = soup.find('h2',class_="title").text.strip()

        #Append Data
        hemispheres["title"] = title
        hemispheres["img"] = img
        hemispheres_list.append(hemispheres)

        #Back to Mars Astropedia
        url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
        browser.visit(url)
    
    # Store data in a dictionary
    data = {
        "article": article,
        "articleurl": articleurl,
        "news_p": news_p,
        "first_image_url": first_image_url,
        "featured_image_url": featured_image_url,
        "mars_weather":mars_weather,
        "marsfacts":mars_facts,
        "hemispheres_list":hemispheres_list
    }

    # Close the browser after scraping
    browser.quit()

    # Return results
    return data

In [4]:
data = scrape_data()

In [5]:
data

{'article': 'Why This Martian Full Moon Looks Like Candy',
 'articleurl': 'https://mars.nasa.gov/news/8436/why-this-martian-full-moon-looks-like-candy/',
 'news_p': "For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.",
 'first_image_url': 'https://jpl.nasa.gov/spaceimages/images/wallpaper/PIA23208-640x350.jpg',
 'featured_image_url': 'https://jpl.nasa.gov/spaceimages/images/wallpaper/PIA14579-1920x1200.jpg',
 'mars_weather': 'InSight sol 159 (2019-05-08) low -100.3ºC (-148.5ºF) high -21.6ºC (-6.9ºF)\nwinds from the SW at 4.6 m/s (10.4 mph) gusting to 15.3 m/s (34.2 mph)\npressure at 7.40 hPapic.twitter.com/rg5UKl8dLc',
 'marsfacts': '<table border="1" class="dataframe">  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>   

In [6]:
data["article"]

'Why This Martian Full Moon Looks Like Candy'

In [7]:
data["articleurl"]

'https://mars.nasa.gov/news/8436/why-this-martian-full-moon-looks-like-candy/'

In [8]:
data["news_p"]

"For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera."

In [9]:
data["first_image_url"]

'https://jpl.nasa.gov/spaceimages/images/wallpaper/PIA23208-640x350.jpg'

In [10]:
data["featured_image_url"]

'https://jpl.nasa.gov/spaceimages/images/wallpaper/PIA14579-1920x1200.jpg'

In [11]:
data["mars_weather"]

'InSight sol 159 (2019-05-08) low -100.3ºC (-148.5ºF) high -21.6ºC (-6.9ºF)\nwinds from the SW at 4.6 m/s (10.4 mph) gusting to 15.3 m/s (34.2 mph)\npressure at 7.40 hPapic.twitter.com/rg5UKl8dLc'

In [12]:
data["marsfacts"]

'<table border="1" class="dataframe">  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [13]:
data["hemispheres_list"][0]["title"]

'Cerberus Hemisphere Enhanced'

In [14]:
data["hemispheres_list"][0]["img"]

'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'